In [16]:
import warnings
warnings.filterwarnings("ignore")

In [19]:
import pandas as pd
import re

In [2]:
#!pip install pyLDAvis
data = pd.read_csv("./submission.csv")
data = pd.DataFrame(data)

In [4]:
data['rubric_id'].unique()

array([10, 15, 32, 38, 44, 61, 65, 72])

In [9]:
rubric_id_10 = data[data["rubric_id"] == 10].reset_index(drop=True)
rubric_id_15 = data[data["rubric_id"] == 15].reset_index(drop=True)
rubric_id_32 = data[data["rubric_id"] == 32].reset_index(drop=True)
rubric_id_38 = data[data["rubric_id"] == 38].reset_index(drop=True)
rubric_id_44 = data[data["rubric_id"] == 44].reset_index(drop=True)
rubric_id_61 = data[data["rubric_id"] == 61].reset_index(drop=True)
rubric_id_65 = data[data["rubric_id"] == 65].reset_index(drop=True)
rubric_id_72 = data[data["rubric_id"] == 72].reset_index(drop=True)
# Print the resulting rows
print("week1",rubric_id_10.shape) #telecom dataset
print("week2",rubric_id_15.shape) #dropped_airflowdb
print("week3",rubric_id_32.shape) #dropped_fewfiles
print("week5",rubric_id_38.shape) #algorand
print("week6",rubric_id_44.shape) #rag
print("week7",rubric_id_61.shape) #amharic finetuning
print("week9",rubric_id_65.shape) #hardhat
print("week10",rubric_id_72.shape) #image generation
#ruburic_id_10.head()

week1 (53, 9)
week2 (40, 9)
week3 (8, 9)
week5 (36, 9)
week6 (35, 9)
week7 (31, 9)
week9 (31, 9)
week10 (32, 9)


In [10]:
week1_python = data[data["rubric_id"] == 10].reset_index(drop=True)
# rubric_id_15 = data[data["rubric_id"] == 15].reset_index(drop=True)
# rubric_id_32 = data[data["rubric_id"] == 32].reset_index(drop=True)
week5_algo = data[data["rubric_id"] == 38].reset_index(drop=True)
week6_rag = data[data["rubric_id"] == 44].reset_index(drop=True)
week7_finetune = data[data["rubric_id"] == 61].reset_index(drop=True)
week9_eth = data[data["rubric_id"] == 65].reset_index(drop=True)
week10_img_generate = data[data["rubric_id"] == 72].reset_index(drop=True)
# Print the resulting rows
print("week1",week1_python.shape) #telecom dataset
# print("week2",rubric_id_15.shape) #dropped_airflowdb
# print("week3",rubric_id_32.shape) #dropped_fewfiles
print("week5",week5_algo.shape) #algorand
print("week6",week6_rag.shape) #rag
print("week7",week7_finetune.shape) #amharic finetuning
print("week9",week9_eth.shape) #hardhat
print("week10",week10_img_generate.shape) #image generation
#ruburic_id_10.head()

week1 (53, 9)
week5 (36, 9)
week6 (35, 9)
week7 (31, 9)
week9 (31, 9)
week10 (32, 9)


In [11]:
week1_python['code_content'].isnull().value_counts()

code_content
False    44
True      9
Name: count, dtype: int64

In [ ]:
# Assuming your DataFrame is named "df" and the column is named "column_name"
week1_python.dropna(subset=['code_content'], inplace=True)

In [13]:
week1_python['code_content'].isnull().value_counts()

code_content
False    44
Name: count, dtype: int64

In [ ]:
# Replace np.nan values with an empty string
week1_python['code_content'].fillna('', inplace=True)
# Drop a column
columns_to_drop = ['submission_id','payload', 'rubric_id', 'rubric_type', 'next_action', 'request_origin', 'params']
week1_python = week1_python.drop(columns_to_drop, axis=1)
print("\nDataFrame after dropping column 'C':")
week1_python.head()

In [18]:
week1_python.head()

,user_id,code_content
0,619,Repository Structure: '\n' ├── top-3 socall me...
1,766,Repository Structure: '\n' ├── ReadMe.md\n├── ...
2,665,Repository Structure: '\n' ├── notebooks\n│ ...
3,606,Repository Structure: '\n' ├── db\n│ ├── sql...
4,699,Repository Structure: '\n' ├── Dashboard\n│ ...


In [20]:
def clean_packages(text):
    patterns = [
    r"### BEGIN File: \w+\.txt ###.*?END File: \w+\.txt ###",
    r"### BEGIN File: (\w+/\w+.txt) ###.*?END File: \1 ###",
    r"### BEGIN File: (\w+/\w+.txt) ###.*?END File: (\w+/\w+.txt) ###",
    r"### BEGIN File: ([\w./-]+\.txt) ###.*?END File: ([\w./-]+\.txt) ###",
    r"### BEGIN File: (\w+/\w+.pgerd) ###.*?END File: (\w+/\w+.pgerd) ###",
    r"### BEGIN File: \w+\.lock ###.*?END File: \w+\.lock ###",
    r"### BEGIN File: \w+\.yml ###.*?END File: \w+\.yml ###",
    r"### BEGIN File: \w+\.yaml ###.*?END File: \w+\.yaml ###",
    r"### BEGIN File: \w+\.toml ###.*?END File: \w+\.toml ###",
    r"### BEGIN File: \w+\.log ###.*?END File: \w+\.log ###",
    #r"### BEGIN File: model_schema\/model_schema_design\.pgerd ###.*?END File: model_schema\/model_schema_design\.pgerd ###",
    r"### BEGIN File: package\.json ###.*?END File: package\.json ###",
    r"### BEGIN File: setup\.py ###.*?END File: setup\.py ###",
    r"### BEGIN File: tailwind.config\.js ###.*?END File: tailwind.config\.js ###",
    r"### BEGIN File: tailwind.config\.ts ###.*?END File: tailwind.config\.ts ###",
    r"### BEGIN File: Dockerfile ###.*?END File: Dockerfile ###",
    ]
    cleaned_text = re.sub("|".join(patterns), '', text, flags=re.DOTALL | re.MULTILINE)
    return cleaned_text
#textt = clean_packages(data['code_content'][151])
week1_python['code_content'] = week1_python['code_content'].apply(clean_packages)

In [21]:
def clean_content(text):
    pattern = r"### BEGIN File: README\.md ###.*?END File: README\.md ###"
    cleaned_text = re.sub(pattern, '', text, flags=re.DOTALL)
    return cleaned_text
#data['code_content'][182] = clean_content(data['code_content'][182])
# Assuming your DataFrame is named 'data' and the column containing the text is named 'code_content'
week1_python['code_content'] = week1_python['code_content'].apply(clean_content)

In [22]:
def clean_License(text):
    pattern = r'BEGIN File: LICENSE.*?END File: LICENSE'
    cleaned_text = re.sub(pattern, '', text, flags=re.DOTALL)
    return cleaned_text
# data['code_content'][182] = clean_License(data['code_content'][182])
# Assuming your DataFrame is named 'data' and the column containing the text is named 'code_content'
week1_python['code_content'] = week1_python['code_content'].apply(clean_License)

In [23]:
def remove_comments(text):
    pattern = r'#.*?\n'
    cleaned_text = re.sub(pattern, '', text, flags=re.DOTALL)
    return cleaned_text
# data['code_content'][182] = remove_comments(data['code_content'][182])
# Assuming your DataFrame is named 'data' and the column containing the text is named 'code_content'
week1_python['code_content'] = week1_python['code_content'].apply(remove_comments)

In [24]:
def clean_perged_files(text):
    pattern = r'### BEGIN File:.*?model_schema/model_schema_design\.pgerd.*?### END File'
    cleaned_text = re.sub(pattern, '', text, flags=re.DOTALL)
    return cleaned_text
# data['code_content'][182] = clean_perged_files(data['code_content'][182])
# Assuming your DataFrame is named 'data' and the column containing the text is named 'code_content'
week1_python['code_content'] = week1_python['code_content'].apply(clean_perged_files)

In [25]:
def extract_content(text):
    pattern = r'Content:\s*(.*?)(?=\s*Commit History:)'
    matches = re.findall(pattern, text, flags=re.DOTALL)
    if matches:
        return '\n'.join(matches)
    else:
        return text
# data['code_content'][182] = extract_content(data['code_content'][182])
# Assuming your DataFrame is named 'df' and the column containing the text is named 'text_column'
week1_python['code_content'] = week1_python['code_content'].apply(extract_content)

In [34]:

def clean_installs(text):
    pattern = r"pip install \w+\n"
    cleaned_text = re.sub(pattern, '', text, flags=re.DOTALL)
    return cleaned_text
# data['code_content'][182] = clean_perged_files(data['code_content'][182])
# Assuming your DataFrame is named 'data' and the column containing the text is named 'code_content'
week1_python['code_content'] = week1_python['code_content'].apply(clean_installs)

In [26]:
column_name = 'code_content'

# Check if the column contains empty strings
has_empty_values = any(week1_python[column_name] == '')

if has_empty_values:
    print("The column contains empty string values.")
else:
    print("The column does not contain empty string values.")

The column does not contain empty string values.


In [36]:
week1_python.head()

,user_id,code_content
0,619,from sqlalchemy import create_engine\nimport p...
1,766,from sqlalchemy import create_engine\n\n\nprin...
2,665,%reload_ext autoreload\n\n%autoreload 2\nimpor...
3,606,import seaborn as sns\nimport matplotlib.pyplo...
4,699,%\n%\n%import pandas as pd\n\nfrom sqlalchemy ...


In [37]:
week1_python.to_csv('./data_files/week1_python.csv', index=False)